In [ ]:
# Custom PPO implementation with Soundspaces 2.0
from configurator import generate_args, get_arg_dict
from th_logger import TBXLogger as TBLogger

# Env deps: Soundspaces and Habitat
from habitat.datasets import make_dataset
from ss_baselines.av_nav.config import get_config
from ss_baselines.common.environments import AudioNavRLEnv
from ss_baselines.common.env_utils import construct_envs
from ss_baselines.common.environments import get_env_class

# Environment config
# TODO: Override some of the config elements through arg parse ?
env_config = get_config(
    config_paths="env_configs/audiogoal_rgb.yaml",
    # run_type="train"
)
# env_config.defrost()
# env_config.TASK_CONFIG.TASK.MEASUREMENTS.append("TOP_DOWN_MAP") # Note: can we add audio sensory info fields here too ?
# env_config.TASK_CONFIG.DATASET.SPLIT = config.EVAL.SPLIT
# env_config.freeze()
# print(env_config)

# dataset = make_dataset(id_dataset=env_config.TASK_CONFIG.DATASET.TYPE,
#                        config=env_config.TASK_CONFIG.DATASET)
# env = AudioNavRLEnv(config=env_config, dataset=dataset)
# print(env)

envs = construct_envs(env_config, get_env_class(env_config.ENV_NAME))
print(envs)
